<font size=7> Sury Cross Correlation Attempt

I attempt to get Sury's code for analyzing connectivity working.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
from scipy import signal
import math
from scipy.sparse import csr_array

In [2]:
import pickle
from human_hip.spike_data import curate_latencies, plot_raster_latency_pairs, plot_vector_layout, plot_latency_dist_hist, plot_latency_angle_hist
from human_hip.animate import animate_latencies

In [3]:
with open("/workspaces/human_hippocampus/data/ephys/2023-12-03-e-Hc112823_avv9hckcr1/curated_base_acqm.pkl", "rb") as filename:
    spike_data = pickle.load(filename)

In [4]:
with open( 'curate_latencies_good_pairs.pkl' , 'rb') as my_file:
    good_pairs = pickle.load(my_file)

# Look at latencies

# <font color="brown">Sury's Code

In [ ]:
def ccg(bt1, bt2, ccg_win=[-10, 10], t_lags_shift=0):
    left_edge, right_edge = np.subtract(ccg_win, t_lags_shift)
    lags = np.arange(ccg_win[0], ccg_win[1] + 1)
    pad_width = min(max(-left_edge, 0), max(right_edge, 0))
    bt2_pad = np.pad(bt2, pad_width=pad_width, mode='constant')
    cross_corr = signal.fftconvolve(bt2_pad, bt1[::-1], mode="valid")
    return np.round(cross_corr), lags


def p_fast(n, lambda_):
    """
    A poisson estimation of the probability of observing n or more events
    """
    ## take log to make sure the factorial does not overflow
    # add poisson_var when x = 0, 1, take log after calculation to avoid log(0)
    if n > 1:
        poisson_01 = [np.exp(-lambda_)*lambda_**x/math.factorial(x) for x in [0, 1]]
        poisson_res = [np.exp(-lambda_ + x*math.log(lambda_) - math.log(math.factorial(x))) for x in range(2, n)]
        poisson_var = poisson_01 + poisson_res
    else:
        poisson_var = [np.exp(-lambda_)*lambda_**x/math.factorial(x) for x in range(n)]
    continuity_correction = np.exp((math.log(0.5) - lambda_ + n*math.log(lambda_)) - math.log(math.factorial(n)))
    return 1 - np.sum(poisson_var) - continuity_correction


def functional_pair(spike_data, binary_bin_size=0.001, ccg_win=[-50, 50],
                    func_latency=5, func_prob=0.00001, verbose=True):
    """
    Note: the input spike times are in seconds!
    """
    train = spike_data["train"]
    neuron_data = spike_data["neuron_data"]
    unit_count = len(train)
    sparse_train = sparse_train(train, bin_size=binary_bin_size)
    
    if unit_count < 2:
        return (0, 0), {}
    for i in range(unit_count-1):
        for j in range(i+1, unit_count):
            counts, lags = ccg(sparse_train[i],
                            sparse_train[j],
                            ccg_win=ccg_win)
            max_ind = np.argmax(counts)
            latency = lags[max_ind]
            if latency >= -func_latency and latency <= func_latency:
                if max_ind != np.diff(ccg_win)//2:
                    ccg_smth = gaussian_filter1d(counts, sigma=10)   
                    # ccg_smth = utils.hollow_gaussian_filter(counts, sigma=10) 
                    lambda_slow_peak = ccg_smth[max_ind]
                    ccg_peak = int(counts[max_ind])
                    # estimate p_fast
                    p_fast_est = p_fast(ccg_peak, lambda_slow_peak)
                    if verbose:
                        print(f"Putative functional pair {i}, {j}")
                        print(f"Cross correlation latency: {latency} ms, counts: {ccg_peak}, smoothed counts: {lambda_slow_peak}")
                        print(f"p_fast: {p_fast_est}")
                    if p_fast_est <= func_prob:    # test with func_prob = 10e-5
                        yield (i, j), {"latency": latency,
                                    "p_fast": p_fast_est,
                                    "ccg": counts,
                                    "lags": lags,
                                    "ccg_smth": ccg_smth}


# test
func_pairs = {}
for (i, j), value in functional_pair():
    lags, counts, ccg_smth = value["lags"], value["ccg"], value["ccg_smth"]
    latency, p_fast = value["latency"], value["p_fast"]
    ccg_peak = np.max(counts)
    fig, axs = plt.subplots(figsize=(5, 3), tight_layout=True)
    plt.suptitle(f"ccg_unit_{i}_{j}")
    axs.bar(lags, counts, label=f"p_fast={p_fast:.3g} \n count={ccg_peak}")
    axs.plot(lags, ccg_smth, color="red")
    axs.axvline(0, linestyle="--", color="black")
    axs.scatter(latency, ccg_peak, color="red", marker="x", label=f"latency={latency}")
    axs.legend()
    axs.set_xlabel("Lags (ms)")
    axs.set_ylabel("Counts")
    plt.savefig(f"ccg_unit_{i}_{j}.png")
    plt.close()
    func_pairs[(i, j)] = value

np.savez("func_pairs.npz", func_pairs=func_pairs)

# <font color="blue"> sparse_train

<font color="blue"> Creates a raster-like matrix where each column is 1 ms. Displays whether or not a spike occured at that time.

<font color="red"> Code creates a matrix that is far to large to be implementable. Trying to find a solution around using it.

In [5]:
def sparse_train(spike_train: list, bin_size=0.001):
    """
    create a sparse matrix for the input spike trains
    with a given bin size
    """
    num = len(spike_train)
    length = np.max([t[-1] for t in spike_train])
    indices = np.hstack([np.ceil(ts / bin_size) - 1
                         for ts in spike_train]).astype(int)
    units = np.hstack([0] + [len(ts) for ts in spike_train])
    indptr = np.cumsum(units)
    values = np.ones_like(indices)
    length = int(np.ceil(length / bin_size))
    np.clip(indices, 0, length - 1, out=indices)
    return csr_array((values, indices, indptr), shape=(num, length)) #.toarray()



# <font color="blue"> CCG

<font color="blue" >computes the cross correlation between two neurons. Returns...

In [6]:
def ccg(bt1, bt2, ccg_win=[-10, 10], t_lags_shift=0):
    left_edge, right_edge = np.subtract(ccg_win, t_lags_shift)
    lags = np.arange(ccg_win[0], ccg_win[1] + 1)
    pad_width = min(max(-left_edge, 0), max(right_edge, 0))
    bt2_pad = np.pad(bt2, pad_width=pad_width, mode='constant')
    cross_corr = signal.fftconvolve(bt2_pad, bt1[::-1], mode="valid")
    return np.round(cross_corr), lags

# <font color="brown"> 1 pair

<font color="brown"> figuring out function

In [7]:
i= good_pairs[0,0]
j= good_pairs[0,1]

In [8]:
train = spike_data.train
neuron_data = spike_data.neuron_data[0]    
unit_count = len(train)
sparse_train = sparse_train(train)

/tmp/ipykernel_30625/1076234412.py:2: DeprecationWarning: Call to deprecated function (or staticmethod) neuron_data. (Use NeuronAttributes instead of neuron_data, with the function load_spike_data())
  neuron_data = spike_data.neuron_data[0]


In [9]:
bt1 = sparse_train[[i],:].toarray()[0]
bt2 = sparse_train[[j],:].toarray()[0] 
ccg_win=[-10, 10]
t_lags_shift=0

In [10]:
left_edge, right_edge = np.subtract(ccg_win, t_lags_shift)
lags = np.arange(ccg_win[0], ccg_win[1] + 1)
pad_width = min(max(-left_edge, 0), max(right_edge, 0))
bt2_pad = np.pad(bt2, pad_width=pad_width, mode='constant')


In [13]:
cross_corr = signal.oaconvolve(bt2_pad, bt1[::-1], mode="valid")

: 

: 

In [12]:
signal.oaconvolve()

<function scipy.signal._signaltools.oaconvolve(in1, in2, mode='full', axes=None)>

In [11]:
del bt2


In [12]:
cross_corr = signal.fftconvolve(bt2_pad, bt1[::-1], mode="valid")

: 

: 

In [ ]:
cross_corr = signal.fftconvolve(bt2_pad, bt1[::-1], mode="valid")

In [47]:
bt1[::-1]

array([0, 0, 0, ..., 0, 0, 0])

In [43]:
bt2[0]

array([0, 0, 0, ..., 0, 0, 0])

In [46]:
bt2_pad

array([0, 0, 0, ..., 0, 0, 0])

In [36]:
counts, lags = ccg( sparse_train[[i],:].toarray(), sparse_train[[j],:].toarray()  )

MemoryError: Unable to allocate 42.7 GiB for an array with shape (21, 273148870) and data type int64

In [32]:
ccg_win=[-10, 10]
t_lags_shift=0
left_edge, right_edge = np.subtract(ccg_win, t_lags_shift) # -10, 10
lags = np.arange(ccg_win[0], ccg_win[1] + 1)  # -10, -9, -8, ..., 9, 10
pad_width = min(max(-left_edge, 0), max(right_edge, 0)) # 10


In [33]:
bt2_pad = np.pad(bt2, pad_width=pad_width, mode='constant')

NameError: name 'bt2' is not defined

# <font color="blue">functional_pair

In [12]:
train = spike_data.train

In [7]:
train = spike_data.train
neuron_data = spike_data.neuron_data[0]    
unit_count = len(train)
sparse_train = sparse_train(train)

/tmp/ipykernel_17876/1076234412.py:2: DeprecationWarning: Call to deprecated function (or staticmethod) neuron_data. (Use NeuronAttributes instead of neuron_data, with the function load_spike_data())
  neuron_data = spike_data.neuron_data[0]


In [15]:
#get a row from the sparse_train matrix
check = sparse_train[[0],:].toarray()

In [19]:
np.sum(check)

3557

In [ ]:

def functional_pair(spike_data, binary_bin_size=0.001, ccg_win=[-50, 50],
                    func_latency=5, func_prob=0.00001, verbose=True):
    """
    Note: the input spike times are in seconds!
    """
    train = spike_data.train
    neuron_data = spike_data.neuron_data[0]    
    unit_count = len(train)
    sparse_train = sparse_train(train, bin_size=binary_bin_size)
    
    if unit_count < 2:
        return (0, 0), {}
    for i in range(unit_count-1):
        for j in range(i+1, unit_count):
            counts, lags = ccg(sparse_train[i],
                            sparse_train[j],
                            ccg_win=ccg_win)
            max_ind = np.argmax(counts)
            latency = lags[max_ind]
            if latency >= -func_latency and latency <= func_latency:
                if max_ind != np.diff(ccg_win)//2:
                    ccg_smth = gaussian_filter1d(counts, sigma=10)   
                    # ccg_smth = utils.hollow_gaussian_filter(counts, sigma=10) 
                    lambda_slow_peak = ccg_smth[max_ind]
                    ccg_peak = int(counts[max_ind])
                    # estimate p_fast
                    p_fast_est = p_fast(ccg_peak, lambda_slow_peak)
                    if verbose:
                        print(f"Putative functional pair {i}, {j}")
                        print(f"Cross correlation latency: {latency} ms, counts: {ccg_peak}, smoothed counts: {lambda_slow_peak}")
                        print(f"p_fast: {p_fast_est}")
                    if p_fast_est <= func_prob:    # test with func_prob = 10e-5
                        yield (i, j), {"latency": latency,
                                    "p_fast": p_fast_est,
                                    "ccg": counts,
                                    "lags": lags,
                                    "ccg_smth": ccg_smth}
